In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import skimage.io
import keras.backend as K
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
from keras.layers import Dense, Flatten, Dropout,BatchNormalization ,Activation
from keras.models import Model, Sequential
from keras.applications.nasnet import NASNetLarge
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from keras.optimizers import Adam

In [2]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255,
                                   validation_split = 0.2,
                                  
        rotation_range=5,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        #zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='nearest')

valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255,
                                  validation_split = 0.2)

test_datagen  = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255
                                  )

In [3]:
train_dir = "../archive/train" #passing the path with training images
test_dir = "../archive/test"   #passing the path with testing images
train_dataset  = train_datagen.flow_from_directory(directory = train_dir,
                                                   target_size = (48,48),
                                                   class_mode = 'categorical',
                                                   subset = 'training',
                                                   batch_size = 64)
valid_dataset = valid_datagen.flow_from_directory(directory = train_dir,
                                                  target_size = (48,48),
                                                  class_mode = 'categorical',
                                                  subset = 'validation',
                                                  batch_size = 64)
test_dataset = test_datagen.flow_from_directory(directory = test_dir,
                                                  target_size = (48,48),
                                                  class_mode = 'categorical',
                                                  batch_size = 64)

Found 22968 images belonging to 7 classes.
Found 5741 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [4]:
base_model = tf.keras.applications.VGG16(input_shape=(48,48,3),include_top=False,weights="imagenet")

In [5]:
for layer in base_model.layers[:-4]:
    layer.trainable=False

In [6]:
# Building Model

model=tf.keras.models.Sequential()
model.add(base_model)
model.add(Dropout(0.5))
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(32,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(32,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(32,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(7,activation='softmax'))

model.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 1, 1, 512)         14714688  
                                                                 
 dropout (Dropout)           (None, 1, 1, 512)         0         
                                                                 
 flatten (Flatten)           (None, 512)               0         
                                                                 
 batch_normalization (BatchN  (None, 512)              2048      
 ormalization)                                                   
                                                                 
 dense (Dense)               (None, 32)                16416     
                                                                 
 batch_normalization_1 (Batc  (None, 32)               128       
 hNormalization)                                        

In [9]:
epochs = 100
batch_size = 64
history=model.fit(train_dataset,validation_data=valid_dataset,epochs = epochs,verbose = 1)

Epoch 1/100
359/359 [==============================] - 20s 52ms/step - loss: 1.2550 - accuracy: 0.5358 - val_loss: 1.3784 - val_accuracy: 0.4963
Epoch 2/100
359/359 [==============================] - 18s 49ms/step - loss: 1.2524 - accuracy: 0.5361 - val_loss: 1.3830 - val_accuracy: 0.4992
Epoch 3/100
359/359 [==============================] - 18s 51ms/step - loss: 1.2594 - accuracy: 0.5368 - val_loss: 1.3351 - val_accuracy: 0.5118
Epoch 4/100
359/359 [==============================] - 18s 51ms/step - loss: 1.2511 - accuracy: 0.5365 - val_loss: 1.3735 - val_accuracy: 0.5086
Epoch 5/100
359/359 [==============================] - 18s 50ms/step - loss: 1.2481 - accuracy: 0.5383 - val_loss: 1.3333 - val_accuracy: 0.5149
Epoch 6/100
359/359 [==============================] - 20s 54ms/step - loss: 1.2494 - accuracy: 0.5399 - val_loss: 1.3359 - val_accuracy: 0.5158
Epoch 7/100
359/359 [==============================] - 31s 88ms/step - loss: 1.2493 - accuracy: 0.5431 - val_loss: 1.3790 - val_ac

In [10]:
model.save_weights('weights/weight2')

In [7]:
model.load_weights('weights/weight2')

In [12]:
def Train_Val_Plot(acc,val_acc,loss,val_loss,auc,val_auc,precision,val_precision,f1,val_f1):
    
    fig, (ax1, ax2,ax3,ax4,ax5) = plt.subplots(1,5, figsize= (20,5))
    fig.suptitle(" MODEL'S METRICS VISUALIZATION ")

    ax1.plot(range(1, len(acc) + 1), acc)
    ax1.plot(range(1, len(val_acc) + 1), val_acc)
    ax1.set_title('History of Accuracy')
    ax1.set_xlabel('Epochs')
    ax1.set_ylabel('Accuracy')
    ax1.legend(['training', 'validation'])


    ax2.plot(range(1, len(loss) + 1), loss)
    ax2.plot(range(1, len(val_loss) + 1), val_loss)
    ax2.set_title('History of Loss')
    ax2.set_xlabel('Epochs')
    ax2.set_ylabel('Loss')
    ax2.legend(['training', 'validation'])

    ax3.plot(range(1, len(auc) + 1), auc)
    ax3.plot(range(1, len(val_auc) + 1), val_auc)
    ax3.set_title('History of AUC')
    ax3.set_xlabel('Epochs')
    ax3.set_ylabel('AUC')
    ax3.legend(['training', 'validation'])
    
    ax4.plot(range(1, len(precision) + 1), precision)
    ax4.plot(range(1, len(val_precision) + 1), val_precision)
    ax4.set_title('History of Precision')
    ax4.set_xlabel('Epochs')
    ax4.set_ylabel('Precision')
    ax4.legend(['training', 'validation'])
    
    ax5.plot(range(1, len(f1) + 1), f1)
    ax5.plot(range(1, len(val_f1) + 1), val_f1)
    ax5.set_title('History of F1-score')
    ax5.set_xlabel('Epochs')
    ax5.set_ylabel('F1 score')
    ax5.legend(['training', 'validation'])


    plt.show()

Train_Val_Plot(history.history['accuracy'],history.history['val_accuracy'],
               history.history['loss'],history.history['val_loss'],
               history.history['auc'],history.history['val_auc'],
               history.history['precision'],history.history['val_precision'],
               history.history['f1_score'],history.history['val_f1_score']
              )

KeyError: 'auc'

In [11]:
# old
loss1, acc1 = model.evaluate(train_dataset)
print("train loss:", loss1, "train accuracy:", acc1)
loss2, acc2 = model.evaluate(valid_dataset)
print("validation loss:", loss2, "validation accuracy:", acc2)
loss3, acc3 = model.evaluate(test_dataset)
print("test loss:", loss3, "test accuracy:", acc3)

359/359 [==============================] - 19s 51ms/step - loss: 1.1180 - accuracy: 0.6280
train loss: 1.1180362701416016 train accuracy: 0.6280041933059692
90/90 [==============================] - 2s 19ms/step - loss: 1.3293 - accuracy: 0.5287
validation loss: 1.3293477296829224 validation accuracy: 0.5286535620689392
113/113 [==============================] - 2s 21ms/step - loss: 1.3098 - accuracy: 0.5376
test loss: 1.3098264932632446 test accuracy: 0.5376149415969849
